In [5]:
import re
import requests
from bs4 import BeautifulSoup

In [6]:
proxies = {
    "http": "http://127.0.0.1:2080",  # Replace with your proxy address
    "https": "http://127.0.0.1:2080"  # For HTTPS requests as well
}


In [7]:
res = requests.get("https://core.telegram.org/bots/api", proxies=proxies)

In [8]:
pattern = r'<h4><a class=\"anchor\" name=\".*?\" href=\".*?\"><i class=\"anchor-icon\"><\/i><\/a>(.*?)<\/h4>\s+<p>(.*?)<\/p>\s+<table class=\"table\">\s+<thead>\s+<tr>\s+<th>Parameter<\/th>\s+<th>Type<\/th>\s+<th>Required<\/th>\s+<th>Description<\/th>\s+<\/tr>\s+<\/thead>\s+<tbody.*?>([\s\S]*?)<\/tbody>'

In [9]:
all_methods = re.findall(pattern, res.text)

In [10]:
table_body_pattern = r'<tr>\s+<td>(.*?)</td>\s+<td>(.*?)</td>\s+<td>(.*?)</td>\s+<td>(.*?)</td>\s+</tr>'

In [11]:
max_char_pattern = r'-(\d+) characters'

In [12]:
def text_cleaner(text):
    return BeautifulSoup(text, 'html.parser').get_text().replace('"', '\\"')

In [19]:
with open('../component/models.py', 'w') as file:
    file.write("""from django.contrib.postgres.fields import ArrayField
from django.db import models


class TelegramComponent(models.Model):
    timestamp = models.DateTimeField(auto_now_add=True)
    class Meta:
        pass


""")

    for i, method in enumerate(all_methods):
        if i < 2:
            continue
        name = method[0]
        comment = method[1]
        body = re.findall(table_body_pattern, method[2])
        file.write(f"class {name[0].upper()+name[1:]}(TelegramComponent):\n")
        file.write(f"    \"\"\"{text_cleaner(comment)}\"\"\"\n\n")

        # print(f"class {name[0].upper()+name[1:]}(TelegramComponent):")
        for item in body:
            type_field = text_cleaner(item[1])

            if item[2] == 'Optional' or item[0] == 'thumbnail':
                django_param = "null=True, blank=True,"
            elif item[2] == 'Yes':
                django_param = "null=False, blank=False,"

            match = re.search(max_char_pattern, text_cleaner(item[3]))
            if match:
                extracted_number = int(match.group(1))  # Convert to integer
                django_param += f" max_length = {extracted_number},"

            django_param += f" help_text=\"{text_cleaner(item[3])}\""

            if type_field == 'String' or type_field == 'Integer or String':
                django_field = f" = models.CharField({django_param})"
            elif type_field == 'Boolean':
                django_field = f" = models.BooleanField({django_param})"
            elif type_field == 'Integer':
                django_field = f" = models.IntegerField({django_param})"
            elif type_field == 'Array of Integer':
                django_field = f"= ArrayField(models.IntegerField(), default=list, {django_param})"
            elif type_field == 'Array of String':
                django_field = f"= ArrayField(models.CharField(), default=list, {django_param})"
            elif type_field == 'Float':
                django_field = f" = models.FloatField({django_param})"
            elif type_field == 'InputFile or String':
                django_field = f" =  models.FileField(upload_to=\"{item[0]}/\", {django_param})"
            else:
                continue

                # type_field = text_cleaner(item[1])
                # if type_field == 'InlineKeyboardMarkup or ReplyKeyboardMarkup or ReplyKeyboardRemove or ForceReply':
                #     pass
                # elif type_field == 'InlineKeyboardMarkup':
                #     pass
                # else:
                #     pass
                # if item[0] in ['reply_markup', 'reply_parameters']:
                #     print( item[0] , text_cleaner(item[1]))


            if django_field:
                file.write(f"    {item[0]}{django_field}\n")
                # print(f"    {item[0]}", django_field)#, f"help_text='{item[3]}'")
                # print("models.CharField")
        # print()
        file.write("\n\n")
file.close()

In [14]:
with open('../component/serializers.py', 'w') as file:
    file.write("""from rest_framework import serializers
from component.models import *


""")
    for i, method in enumerate(all_methods):
        if i < 2:
            continue
        name = method[0]
        comment = method[1]
        file.write(f"class {name[0].upper()+name[1:]}Serializer(serializers.ModelSerializer):\n")
        file.write(f"    class Meta:\n")
        file.write(f"        model = {name[0].upper()+name[1:]}\n")
        file.write(f"        depth = 1\n")
        file.write(f"        fields = \"__all__\"")
        file.write("\n\n")
file.close()

In [15]:
with open('../component/views.py', 'w') as file:
    file.write("""from rest_framework.viewsets import ModelViewSet
from component.serializers import *


""")
    for i, method in enumerate(all_methods):
        if i < 2:
            continue
        name = method[0]
        comment = method[1]
        correct_name = name[0].upper()+name[1:]
        file.write(f"class {correct_name}ViewSet(ModelViewSet):\n")
        file.write(f"    serializer_class = {correct_name}Serializer\n")
        file.write(f"    queryset = {correct_name}.objects.all()\n")
        file.write("\n\n")
file.close()

In [16]:
with open('../component/urls.py', 'w') as file:
    file.write("""from django.urls import include, path
from rest_framework.routers import DefaultRouter

from component.views import *

router = DefaultRouter()

""")
    for i, method in enumerate(all_methods):
        if i < 2:
            continue
        name = method[0]
        comment = method[1]
        correct_name = name[0].upper()+name[1:]
        under_lined_name = ""
        for char in name:
            if char.isupper():
                under_lined_name += "-"+char.lower()
            else:
                under_lined_name += char

        file.write(f"router.register(r\"{under_lined_name}\", {correct_name}ViewSet)\n")

    file.write("""

urlpatterns = [
    path("", include(router.urls)),
]
""")
file.close()

In [17]:
!python -m black ../component/*

reformatted ../component/urls.py
reformatted ../component/views.py
reformatted ../component/serializers.py
reformatted ../component/models.py

All done! ✨ 🍰 ✨
4 files reformatted, 6 files left unchanged.


In [18]:
!pre-commit run --all-files

isort....................................................................Failed
- hook id: isort
- files were modified by this hook

Fixing /home/ali/Documents/nocode/backend/component/serializers.py
Fixing /home/ali/Documents/nocode/backend/component/views.py

black....................................................................Passed
Add trailing commas......................................................Failed
- hook id: add-trailing-comma
- exit code: 1
- files were modified by this hook

Rewriting component/urls.py
Rewriting component/models.py

trim trailing whitespace.................................................Passed
mypy.....................................................................Failed
- hook id: mypy
- files were modified by this hook

Success: no issues found in 42 source files

